In [ ]:
#Copying JL's code, testing new regression model

In [28]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [29]:
#parse data
df_year_salary_exp_qual = pd.read_csv("Teacher_Year_Salary_Experience_Qualification.csv")
df_state_2011_salary_exp = pd.read_csv("Teacher_State_2011_Salary_Experience.csv")
df_merged_overall = pd.read_csv("merged.csv")

# Create a new column "Total enrolment"
df_merged_overall['Total enrolment'] = df_merged_overall['grade_9_students_enrollment'] + df_merged_overall['grade_10_students_enrollment'] + df_merged_overall['grade_11_students_enrollment'] + df_merged_overall['grade_12_students_enrollment']

# Create a new column "SAT mean score"
df_merged_overall['SAT mean score'] = df_merged_overall['critical_reading_mean'] + df_merged_overall['mathematics_mean'] + df_merged_overall['writing_mean']

df_merged_overall.head()

,state_name,year,secondary_teachers,total_rd_expenditures,pupil_teacher_ratio,full_time_equivalent,grade_9_students_enrollment,grade_10_students_enrollment,grade_11_students_enrollment,grade_12_students_enrollment,grade_9_dropouts,grade_10_dropouts,grade_11_dropouts,grade_12_dropouts,grades_9_12_dropouts,no_of_sat_test_takers,critical_reading_mean,critical_reading_sd,mathematics_mean,mathematics_sd,writing_mean,writing_sd,no_high_school_diploma,high_school_or_equivalent,some_college_less_than_4_yr_degree,bachelors_degree_or_higher,american_indian_alaska_native_male_grade_9,american_indian_alaska_native_female_grade_9,asian_or_asian_pacific_islander_male_grade_9,asian_or_asian_pacific_islander_female_grade_9,hispanic_male_grade_9,hispanic_female_grade_9,black_male_grade_9,black_female_grade_9,white_male_grade_9,white_female_grade_9,american_indian_alaska_native_male_grade_10,american_indian_alaska_native_female_grade_10,asian_or_asian_pacific_islander_male_grade_10,asian_or_asian_pacific_islander_female_grade_10,hispanic_male_grade_10,hispanic_female_grade_10,black_male_grade_10,black_female_grade_10,white_male_grade_10,white_female_grade_10,american_indian_alaska_native_male_grade_11,american_indian_alaska_native_female_grade_11,asian_or_asian_pacific_islander_male_grade_11,asian_or_asian_pacific_islander_female_grade_11,hispanic_male_grade_11,hispanic_female_grade_11,black_male_grade_11,black_female_grade_11,white_male_grade_11,white_female_grade_11,american_indian_alaska_native_male_grade_12,american_indian_alaska_native_female_grade_12,asian_or_asian_pacific_islander_male_grade_12,asian_or_asian_pacific_islander_female_grade_12,hispanic_male_grade_12,hispanic_female_grade_12,black_male_grade_12,black_female_grade_12,white_male_grade_12,white_female_grade_12,Public Grads,Private Grads,Total_grads,Total enrolment,SAT mean score
0,ALABAMA,2013,14133.06,NaN,15.82,47161.77,62069,56144,51527,48965,864,1128,1048,862,3902,3172,547,119,538,122,532,118,256,615,496,550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,218705,1617
1,ALABAMA,2003,19918.00,557859,12.60,58070.00,62718,53695,47489,42005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,381,570,448,414,280,229,300,257,554,492,12131,11205,19626,17551,218,231,236,238,416,374,9802,9788,16536,15796,242,227,264,227,292,259,8064,8574,14919,14371,193,175,195,219,205,213,6604,7527,13513,13127,36464,5054,41518,205907,NaN
2,ALABAMA,2012,21449.70,NaN,14.35,51877.22,61642,56229,50387,48945,865,900,829,698,3292,3558,544,117,534,121,530,118,238,588,553,504,280,295,401,362,1281,1152,11627,10708,18304,16776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44400,4708,49108,217203,1608
3,ALASKA,2003,2329.00,140641,17.20,7808.00,11803,10623,9161,8651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,66,83,54,1678,1643,356,320,200,196,241,237,3587,3345,1308,1271,363,353,175,171,255,231,3413,3081,1032,999,323,314,182,137,199,160,2980,2835,921,873,310,324,148,136,206,183,2888,2660,7236,305,7541,40238,NaN
4,ALABAMA,2011,20759.19,NaN,15.60,47722.67,61412,55908,50785,49510,1256,1335,1221,842,4654,3879,528,121,531,121,527,116,310,564,562,427,269,250,387,373,1165,1023,11432,10461,18120,16906,239,281,372,372,1025,886,9904,9774,16863,15734,264,251,315,348,832,753,8403,8820,15516,14862,249,219,324,313,712,649,8049,9126,14905,14515,44570,4726,49296,217615,1586


In [30]:
# Duplicate the dataframe for refinement
df_normalized_merged_overall = df_merged_overall

# Generate the list of column names to be normalized
list_column = list(df_merged_overall.columns.values)
exclude_list = ['state_name', 'year', 'pupil_teacher_ratio', 
                'critical_reading_mean', 'critical_reading_sd', 
                'mathematics_mean', 'mathematics_sd', 'writing_mean', 
                'writing_sd', 'Total enrollment', 'SAT mean score']

# Normalizing with number of enrollments of each state
for col_name in list_column:
    if col_name not in exclude_list:
        try: df_normalized_merged_overall[col_name] = df_normalized_merged_overall[col_name] / df_merged_overall['Total enrollment']
        except: 
            print "warning non numerical value", col_name
    
print df_normalized_merged_overall.shape
df_normalized_merged_overall.head()

warning non numerical value secondary_teachers
warning non numerical value total_rd_expenditures
warning non numerical value full_time_equivalent
warning non numerical value grade_9_students_enrollment
warning non numerical value grade_10_students_enrollment
warning non numerical value grade_11_students_enrollment
warning non numerical value grade_12_students_enrollment
warning non numerical value grade_9_dropouts
warning non numerical value grade_10_dropouts
warning non numerical value grade_11_dropouts
warning non numerical value grade_12_dropouts
warning non numerical value grades_9_12_dropouts
warning non numerical value no_of_sat_test_takers
warning non numerical value no_high_school_diploma
warning non numerical value high_school_or_equivalent
warning non numerical value some_college_less_than_4_yr_degree
warning non numerical value bachelors_degree_or_higher
warning non numerical value american_indian_alaska_native_male_grade_9
warning non numerical value american_indian_alaska_

,state_name,year,secondary_teachers,total_rd_expenditures,pupil_teacher_ratio,full_time_equivalent,grade_9_students_enrollment,grade_10_students_enrollment,grade_11_students_enrollment,grade_12_students_enrollment,grade_9_dropouts,grade_10_dropouts,grade_11_dropouts,grade_12_dropouts,grades_9_12_dropouts,no_of_sat_test_takers,critical_reading_mean,critical_reading_sd,mathematics_mean,mathematics_sd,writing_mean,writing_sd,no_high_school_diploma,high_school_or_equivalent,some_college_less_than_4_yr_degree,bachelors_degree_or_higher,american_indian_alaska_native_male_grade_9,american_indian_alaska_native_female_grade_9,asian_or_asian_pacific_islander_male_grade_9,asian_or_asian_pacific_islander_female_grade_9,hispanic_male_grade_9,hispanic_female_grade_9,black_male_grade_9,black_female_grade_9,white_male_grade_9,white_female_grade_9,american_indian_alaska_native_male_grade_10,american_indian_alaska_native_female_grade_10,asian_or_asian_pacific_islander_male_grade_10,asian_or_asian_pacific_islander_female_grade_10,hispanic_male_grade_10,hispanic_female_grade_10,black_male_grade_10,black_female_grade_10,white_male_grade_10,white_female_grade_10,american_indian_alaska_native_male_grade_11,american_indian_alaska_native_female_grade_11,asian_or_asian_pacific_islander_male_grade_11,asian_or_asian_pacific_islander_female_grade_11,hispanic_male_grade_11,hispanic_female_grade_11,black_male_grade_11,black_female_grade_11,white_male_grade_11,white_female_grade_11,american_indian_alaska_native_male_grade_12,american_indian_alaska_native_female_grade_12,asian_or_asian_pacific_islander_male_grade_12,asian_or_asian_pacific_islander_female_grade_12,hispanic_male_grade_12,hispanic_female_grade_12,black_male_grade_12,black_female_grade_12,white_male_grade_12,white_female_grade_12,Public Grads,Private Grads,Total_grads,Total enrolment,SAT mean score
0,ALABAMA,2013,14133.06,NaN,15.82,47161.77,62069,56144,51527,48965,864,1128,1048,862,3902,3172,547,119,538,122,532,118,256,615,496,550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,218705,1617
1,ALABAMA,2003,19918.00,557859,12.60,58070.00,62718,53695,47489,42005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,381,570,448,414,280,229,300,257,554,492,12131,11205,19626,17551,218,231,236,238,416,374,9802,9788,16536,15796,242,227,264,227,292,259,8064,8574,14919,14371,193,175,195,219,205,213,6604,7527,13513,13127,36464,5054,41518,205907,NaN
2,ALABAMA,2012,21449.70,NaN,14.35,51877.22,61642,56229,50387,48945,865,900,829,698,3292,3558,544,117,534,121,530,118,238,588,553,504,280,295,401,362,1281,1152,11627,10708,18304,16776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44400,4708,49108,217203,1608
3,ALASKA,2003,2329.00,140641,17.20,7808.00,11803,10623,9161,8651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,66,83,54,1678,1643,356,320,200,196,241,237,3587,3345,1308,1271,363,353,175,171,255,231,3413,3081,1032,999,323,314,182,137,199,160,2980,2835,921,873,310,324,148,136,206,183,2888,2660,7236,305,7541,40238,NaN
4,ALABAMA,2011,20759.19,NaN,15.60,47722.67,61412,55908,50785,49510,1256,1335,1221,842,4654,3879,528,121,531,121,527,116,310,564,562,427,269,250,387,373,1165,1023,11432,10461,18120,16906,239,281,372,372,1025,886,9904,9774,16863,15734,264,251,315,348,832,753,8403,8820,15516,14862,249,219,324,313,712,649,8049,9126,14905,14515,44570,4726,49296,217615,1586


In [4]:
#Subset from 2004 onwards

df_merged_2004_onwards = df_normalized_merged_overall[df_normalized_merged_overall['year'] >= 2004]
print df_merged_2004_onwards.shape
df_merged_2004_onwards.head()

(510, 15)


,state_name,year,secondary_teachers,grade_9_students_enrollment,grade_10_students_enrollment,grade_11_students_enrollment,grade_12_students_enrollment,grade_9_dropouts,grade_10_dropouts,grade_11_dropouts,grade_12_dropouts,grades_9_12_dropouts,no_of_sat_test_takers,Total enrolment,SAT mean score
0,ALABAMA,2013,0.064622,0.283802,0.256711,0.235600,0.223886,0.003951,0.005158,0.004792,0.003941,0.017841,3172,218705,1617
2,ALABAMA,2012,0.098754,0.283799,0.258878,0.231981,0.225342,0.003982,0.004144,0.003817,0.003214,0.015156,3558,217203,1608
4,ALABAMA,2011,0.095394,0.282205,0.256912,0.233371,0.227512,0.005772,0.006135,0.005611,0.003869,0.021386,3879,217615,1586
6,ALABAMA,2010,0.096001,0.282126,0.259728,0.234126,0.224020,0.005479,0.006349,0.005659,0.004943,0.022429,3664,221940,1623
8,ALABAMA,2009,0.093018,0.288562,0.263304,0.228702,0.219431,0.005754,0.006656,0.006187,0.005772,0.024370,3365,219495,1650


In [5]:
list_states = pd.unique(df_merged_2004_onwards['state_name'].ravel())
print list_states

['ALABAMA' 'ALASKA' 'ARIZONA' 'ARKANSAS' 'CALIFORNIA' 'COLORADO'
 'CONNECTICUT' 'DELAWARE' 'DISTRICT OF COLUMBIA' 'FLORIDA' 'GEORGIA'
 'HAWAII' 'IDAHO' 'ILLINOIS' 'INDIANA' 'IOWA' 'KANSAS' 'KENTUCKY'
 'LOUISIANA' 'MAINE' 'MARYLAND' 'MASSACHUSETTS' 'MICHIGAN' 'MINNESOTA'
 'MISSISSIPPI' 'MISSOURI' 'MONTANA' 'NEBRASKA' 'NEVADA' 'NEW HAMPSHIRE'
 'NEW JERSEY' 'NEW MEXICO' 'NEW YORK' 'NORTH CAROLINA' 'NORTH DAKOTA'
 'OHIO' 'OKLAHOMA' 'OREGON' 'PENNSYLVANIA' 'RHODE ISLAND' 'SOUTH CAROLINA'
 'SOUTH DAKOTA' 'TENNESSEE' 'TEXAS' 'UTAH' 'VERMONT' 'VIRGINIA'
 'WASHINGTON' 'WEST VIRGINIA' 'WISCONSIN' 'WYOMING']


In [6]:
# X_data
X_data = df_merged_2004_onwards.copy()

# Update the state_name to index to allow for sklearn k-fold cross validation
X_data['state_name'] = X_data['state_name'].apply(lambda r : list_states.tolist().index(r))

# Remove row that have NaN
X_data.dropna(axis=0, how='any', inplace=True)

# Y_response
Y_response = X_data['no_of_sat_test_takers']

# Drop the Y_response
X_data = X_data.drop('no_of_sat_test_takers', 1)

print X_data.shape
X_data.head()

(433, 14)


,state_name,year,secondary_teachers,grade_9_students_enrollment,grade_10_students_enrollment,grade_11_students_enrollment,grade_12_students_enrollment,grade_9_dropouts,grade_10_dropouts,grade_11_dropouts,grade_12_dropouts,grades_9_12_dropouts,Total enrolment,SAT mean score
0,0,2013,0.064622,0.283802,0.256711,0.235600,0.223886,0.003951,0.005158,0.004792,0.003941,0.017841,218705,1617
2,0,2012,0.098754,0.283799,0.258878,0.231981,0.225342,0.003982,0.004144,0.003817,0.003214,0.015156,217203,1608
4,0,2011,0.095394,0.282205,0.256912,0.233371,0.227512,0.005772,0.006135,0.005611,0.003869,0.021386,217615,1586
6,0,2010,0.096001,0.282126,0.259728,0.234126,0.224020,0.005479,0.006349,0.005659,0.004943,0.022429,221940,1623
8,0,2009,0.093018,0.288562,0.263304,0.228702,0.219431,0.005754,0.006656,0.006187,0.005772,0.024370,219495,1650


In [7]:
Y_response.head()

0    3172
2    3558
4    3879
6    3664
8    3365
Name: no_of_sat_test_takers, dtype: float64

In [8]:
# Subset df_merged_overall by state
df_2004_onwards = {}
for state in list_states:
    df_2004_onwards[state] = pd.DataFrame(df_merged_2004_onwards[df_merged_2004_onwards['state_name'] == state])

### Exploratory Data Analysis (EDA) 

In [9]:
#def cv_optimize(clf, parameters, X, y, n_folds=5, score_func='None'):
    #gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds)
    #gs.fit(X, y)
    #print "BEST PARAMS", gs.best_params_
    #best = gs.best_estimator_
    #return best

In [26]:
from sklearn import cross_validation
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.grid_search import GridSearchCV

#clf = LinearRegression()
#clf =LogisticRegression(penalty="l1")

clf = linear_model.Lasso(alpha=10)

scores = cross_validation.cross_val_score(clf, X_data, Y_response, cv=5)
print "Cross validation scores", scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))



Cross validation scores [ 0.96967562  0.80110093  0.60254143  0.7792811   0.90640969]
Accuracy: 0.81 (+/- 0.25)


In [17]:
parameters = {"alpha": [0.0001, 0.001, 0.1, 1, 10, 100]}
gs = GridSearchCV(clf, param_grid=parameters, cv=5)
gs.fit(X_data, Y_response)
print "BEST PARAMS", gs.best_params_
best = gs.best_estimator_
print best

BEST PARAMS {'alpha': 10}
Lasso(alpha=10, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)


In [ ]:
clfsvm, X_train, Y_train, X_test, Y_test = do_classify(linear_model.Lasso(), {"alpha": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse,lcols, u'RESP',1, mask=mask)

In [11]:
#Exploratory Data Analysis (EDA) - merged.csv

#from sklearn.linear_model import LogisticRegression

#clf = LinearRegression()

#clflog, Xtrain, ytrain, Xtest, ytest = do_classify(LogisticRegression(penalty="l1"), {"C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse, lcols, u'RESP',1, mask=mask, reuse_split=reuse_split)


#clf = LinearRegression()
#scores = cross_validation.cross_val_score(clf, X_data, Y_response, cv=5)
#print "Cross validation scores", scores
#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
